In [40]:
import pandas as pd
import os
#cerrega todo os arquivos
dfsNew = []
dfsOld = []

for arq in os.listdir("./data/PT/"):
    if arq.find("New") > 0:
        dfsNew.append(pd.read_csv("./data/PT/"+arq, sep="\t", index_col=0))
    elif arq.find("Old") > 0:
        dfsOld.append(pd.read_csv("./data/PT/"+arq, sep="\t", index_col=0))



In [41]:
from nltk import ngrams
from sacremoses import MosesTokenizer

def tokenize(text, lang, nGram=3):
    entok = MosesTokenizer(lang=lang)
    text = entok.tokenize(text, escape=False)
    grams = []
    for i in range(1, nGram):
        i_grams = [
            " ".join(gram)
            for gram in ngrams(text, i)
        ]
        grams.extend(i_grams)
        
    return grams

def getNgramOverlap(hypothesys, references, nGram, lang):

  overlaps = []
  for h, r in zip([hypothesys], [references]):
    if (h == "") or (r == ""):
      overlaps.append(1.0)
      continue
    a = tokenize(h, lang, nGram)
    b = tokenize(r, lang, nGram)

    if len(a) >= len(b):
      overlaps.append(len(set(a) & set(b))/len(a))
    elif len(b) >= len(a):
      overlaps.append(len(set(a) & set(b))/len(b))

  return overlaps[0]

import numpy as np

def getStats(dfA, dfB, lang):
    df = dfA.merge(dfB, on=["livro", "capitulo", "versiculo"])

    df["sourceLen"] = df["texto_x"].apply(lambda x: getSizeSentece(x))
    df["targetLen"] = df["texto_y"].apply(lambda x: getSizeSentece(x))
    
    df["overlap"] = df.apply(lambda x: getNgramOverlap(x["texto_x"], x["texto_y"], 3, lang), axis=1)

    return df
#Matriz do novo testamento

def getSizeSentece(text):
    try:
        return len(text.split(" "))
    except:
        return 0
    


In [42]:
len(dfsOld)

5

In [43]:
print("Novo testamento: ")
import copy
concat_df_new = []
new = copy.deepcopy(dfsNew)
for dfA in new:
    new = new[1:]
    for dfB in new:
             
        print(set(dfA["livro"]) - set(dfB["livro"]))
        concat_df_new.append(getStats(dfA, dfB, "pt"))


Novo testamento: 
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()


In [44]:
dfConcat_novo = pd.concat(concat_df_new, ignore_index=False)
print("tamanho sem cortes: ", dfConcat_novo.shape)

dfConcat_novo["VERSAO"] = "NOVO"

tamanho sem cortes:  (115364, 10)


In [45]:
print("Velho testamento: ")
import copy
concat_df_old = []
old = copy.deepcopy(dfsOld)
for dfA in old:
    old = old[1:]
    for dfB in old:
             
        print(set(dfA["livro"]) - set(dfB["livro"]))
        concat_df_old.append(getStats(dfA, dfB, "pt"))

Velho testamento: 
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()


In [46]:
dfConcat_old = pd.concat(concat_df_old, ignore_index=False)
print("tamanho sem cortes: ", dfConcat_old.shape)
dfConcat_old["VERSAO"] = "VELHO"

tamanho sem cortes:  (223546, 10)


In [47]:
df_geral = pd.concat([dfConcat_old,dfConcat_novo])

print(df_geral["sourceLen"].mean())
print(df_geral["targetLen"].mean())
print(df_geral["overlap"].mean())

27.371939452952113
27.29075860848013
0.2873294570261173


In [48]:
df_geral.dropna(inplace=True)
df_geral = df_geral[(df_geral.sourceLen >= 5) & (df_geral.targetLen >= 5) ]

In [49]:
selRows = df_geral[(df_geral["livro"] == "Mateus") & (df_geral["capitulo"] == 1)  & (df_geral["versiculo"] >= 1) & (df_geral["versiculo"] <= 16)].index
df_geral.drop(selRows, axis=0,inplace=True)

selRows = df_geral[(df_geral["livro"] == "Lucas") & (df_geral["capitulo"] == 3)  & (df_geral["versiculo"] >= 23) & (df_geral["versiculo"] <= 38)].index
df_geral.drop(selRows, axis=0,inplace=True)

selRows = df_geral[(df_geral["livro"] == "Gênesis") & (df_geral["capitulo"] == 5)  & (df_geral["versiculo"] >= 1) & (df_geral["versiculo"] <= 32)].index
df_geral.drop(selRows, axis=0,inplace=True)

selRows = df_geral[(df_geral["livro"] == "Gênesis") & (df_geral["capitulo"] == 10)  & (df_geral["versiculo"] >= 1) & (df_geral["versiculo"] <= 32)].index
df_geral.drop(selRows, axis=0,inplace=True)


filter = df_geral['livro'].str.contains("1%20Crônicas")
df_geral = df_geral[~filter]
filter = df_geral['livro'].str.contains("2%20Crônicas")
df_geral = df_geral[~filter]


filter = df_geral['texto_x'].str.contains("Copyright")
df_geral = df_geral[~filter]
filter = df_geral['texto_x'].str.contains("copyright")
df_geral = df_geral[~filter]
filter = df_geral['texto_x'].str.contains("®")
df_geral = df_geral[~filter]

filter = df_geral['texto_y'].str.contains("Copyright")
df_geral = df_geral[~filter]
filter = df_geral['texto_y'].str.contains("copyright")
df_geral = df_geral[~filter]
filter = df_geral['texto_y'].str.contains("®")
df_geral = df_geral[~filter]



print(df_geral["sourceLen"].mean())
print(df_geral["targetLen"].mean())
print(df_geral["overlap"].mean())

27.368023053743702
27.1239104980243
0.2879330615451806


In [50]:
df_geral

,estilo_x,livro,capitulo,versiculo,texto_x,estilo_y,texto_y,sourceLen,targetLen,overlap,VERSAO
17,ARC,Gênesis,1,18,"e para governar o dia e a noite , e para fazer...",NTLH,para governarem o dia e a noite e para separar...,27,26,0.433962,VELHO
18,ARC,Gênesis,1,19,E foi a tarde e a manhã : o dia quarto .,NTLH,"A noite passou , e veio a manhã . Esse foi o q...",12,15,0.310345,VELHO
19,ARC,Gênesis,1,20,E disse Deus : Produzam as águas abundantement...,NTLH,Depois Deus disse : — Que as águas fiquem chei...,25,28,0.181818,VELHO
20,ARC,Gênesis,1,21,"E Deus criou as grandes baleias , e todo répti...",NTLH,"Assim Deus criou os grandes monstros do mar , ...",40,45,0.235955,VELHO
21,ARC,Gênesis,1,22,"E Deus os abençoou , dizendo : Frutificai , e ...",NTLH,Ele abençoou os seres vivos do mar e disse : —...,27,31,0.311475,VELHO
...,...,...,...,...,...,...,...,...,...,...,...
7587,OL,Apocalipse,22,16,"“ Eu , Jesus , enviei o meu anjo para vos reve...",VFL,"—Eu , Jesus , lhes enviei o meu anjo para test...",38,32,0.400000,NOVO
7588,OL,Apocalipse,22,17,O Espírito Santo e a esposa dizem : “ Vem ! ” ...,VFL,O Espírito e a noiva dizem : —Venha ! E aquele...,45,38,0.370787,NOVO
7589,OL,Apocalipse,22,18,Eu declaro solenemente a todo aquele que ouvir...,VFL,"Eu , João , aviso solenemente aos que ouvem as...",33,33,0.384615,NOVO
7590,OL,Apocalipse,22,19,"E se alguém tirar , seja o que for , às palavr...",VFL,E se alguém tirar qualquer coisa das palavras ...,37,32,0.465753,NOVO


In [52]:
df_geral.to_csv("./data/FILTERED/portuguese.tsv", sep="\t")